In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import importlib
import os
import matplotlib.patches as patches
import dask
from dask.distributed import Client, LocalCluster
from dask_jobqueue import SGECluster
import socket
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, MaxPool2D, Conv2D, Flatten

In [3]:
hostname = socket.gethostname()
n_workers = 1
n_cores = 4

wks2 = "wn-wks2.fe.hhi.de"
gpu1 = "wn-gpu1.fe.hhi.de"
gpu2 = "wn-gpu-104-01.fe.hhi.de"

if hostname == wks2:
    path = "/data/cluster/projects/infineon-radar/daq_x-har/3_Walking_converted/recording-2020-01-28_11-31-55"
    mem = "20G"      # Allocated memory is critical. For this example it must be at least 16GB 
    q = "wn-37.q"    # Check current queue status on https://hpc-management.fe.hhi.de/wn/phpqstat/
    
    cluster = SGECluster(n_workers=n_workers, cores=n_cores, memory=mem,
                        resource_spec=f"h_vmem={mem}", host=hostname, queue=q,
                         job_extra=["-v MKL_NUM_THREADS=1,NUMEXPR_NUM_THREADS=1,OMP_NUM_THREADS=1"])
elif hostname in (gpu1, gpu2):
    path = os.getcwd() + "/data"
    os.environ["CUDA_VISIBLE_DEVICES"]="0,3"  # Check current status with nvidia-smi and pick GPU from 0-3
    cluster = LocalCluster(n_workers=n_workers, threads_per_worker=n_cores, host=hostname)
else:
    raise ValueError(f"{hostname} is not a supported host. Please run this example on {wks}, {gpu1} or {gpu2}.")
    

    
client = Client(cluster)
client.wait_for_workers(n_workers=n_workers)
client

Client Scheduler: tcp://10.66.20.3:33322 Dashboard: http://10.66.20.3:8787/status,Cluster Workers: 1 Cores: 4 Memory: 540.49 GB


In [36]:
def split_dataset(dataset: tf.data.Dataset, validation_data_fraction: float):

    validation_data_percent = round(validation_data_fraction * 100)
    if not (0 <= validation_data_percent <= 100):
        raise ValueError("validation data fraction must be ∈ [0,1]")

    dataset = dataset.enumerate()
    train_dataset = dataset.filter(lambda f, data: f % 100 >= validation_data_percent)
    validation_dataset = dataset.filter(lambda f, data: f % 100 < validation_data_percent)

    # remove enumeration
    train_dataset = train_dataset.map(lambda f, data: data)
    validation_dataset = validation_dataset.map(lambda f, data: data)

    return train_dataset, validation_dataset

def data_prep(path):
    dataset_raw = np.load(path, allow_pickle=True)
    arrays = np.array(dataset_raw[()]["data"])
    arrays = arrays - arrays.min()
    arrays = arrays/arrays.max()
    arrays -= arrays.mean()
    arrays = arrays/arrays.std()
    if np.isnan(arrays).any() or np.isinf(arrays).any():
        raise "data have imperfections"
    print(arrays.shape)
    labels = dataset_raw[()]["label"]
    labels = np.array([x-np.array(list(set(labels))).min() for x in labels])
    print(labels.shape)
    return (arrays, labels)

def load_data(path):
    data, label = data_prep(path)
    dataset = tf.data.Dataset.from_tensor_slices((data, label))
    dataset = dataset.shuffle(10000)
    train_dataset, rest = split_dataset(dataset, 0.2)
    test_dataset, valid_dataset = split_dataset(rest, 0.5)
    train_data = train_dataset.shuffle(1000).batch(10)
    valid_data = valid_dataset.batch(10)
    test_data = test_dataset.batch(10)
    return train_data, valid_data, test_data


def make_model():
    model = tf.keras.Sequential([
    Input((1,30,30)),
    Conv2D(filters = 8, kernel_size=(3,3), padding="same", activation="relu", name="c1", data_format="channels_first"),
    Conv2D(filters = 16, kernel_size=(3,3), padding="same", activation="relu", name="c2", data_format="channels_first"),
    MaxPool2D(pool_size=(2,2), strides=(1,1),padding="same", name="m1", data_format="channels_first"),
    
    Conv2D(filters = 16, kernel_size=(3,3), padding="same", activation="relu", name="c3", data_format="channels_first"),
    MaxPool2D(pool_size=(2,2), strides=(2,2),padding="valid", name="m2",data_format="channels_first"),
    
    Flatten(),
    Dense(512, activation="relu", use_bias=False),
    Dense(5,  use_bias=False)])
    return model



In [37]:
train_data, valid_data, test_data = load_data("/home/fe/khodabakhshandeh/Projects/radar/radar-ml/Python/box_data.npy")

(59003, 1, 30, 30)
(59003,)


In [39]:
#model = make_model()
print(model.summary())
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
model.fit(train_data, verbose=1, validation_data=valid_data, epochs=5)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
c1 (Conv2D)                  (None, 8, 30, 30)         80        
_________________________________________________________________
c2 (Conv2D)                  (None, 16, 30, 30)        1168      
_________________________________________________________________
m1 (MaxPooling2D)            (None, 16, 30, 30)        0         
_________________________________________________________________
c3 (Conv2D)                  (None, 16, 30, 30)        2320      
_________________________________________________________________
m2 (MaxPooling2D)            (None, 16, 15, 15)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3600)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)              

In [43]:
# model.save("base_model")
model.evaluate(test_data)

    590/Unknown - 5s 8ms/step - loss: 0.6584 - accuracy: 0.7268

[0.6584232585615147, 0.72677964]

In [56]:
model = tf.keras.models.load_model("base_model")

In [64]:
def select_random_data(train_data):
    train_data.shuffle(1000)
    _, data = split_dataset(train_data, 0.1)
    return data

def UCB1(mu, n, t):
    P = np.sqrt(2*np.log10(t)/n)
    index = np.argmax(np.add(mu, P))
    return index

In [61]:
T = 1024
W = model.layers[-2].get_weights()[0]
# n = np.zeros((W.shape[-1]))
# mu = np.zeros_like(n)
threshold = 0.005
for i in range(1, T):
    #select random train data for comparison
    data = select_random_data(train_data)
    
    #selecting index exploration/exploitation
    if np.where(n==0)[0].size == 0:
        index = UCB1(mu, n, i)
        print("RL")
    else:
        index = np.where(n==0)[0][0]
    
    print(index)
    
    #evaluating main model
    loss_base = model.evaluate(data)[0]
    print("/n")

    
    #setting selected node to zero and evaluating again
    W_ = np.copy(W)
    W_[:, index] = 0
    model.layers[-2].set_weights([W_])
    loss = model.evaluate(data)[0]
    print("/n")
    
    #calculating delta and reward
    delta = loss_base - loss
    reward = max(0, threshold + delta)
    print(reward)
    
    #updating number of visiting the node and the average reward
    n[index] +=1
    mu[index] = ((n[index]-1)/n[index])*mu[index] + (1/n[index])*reward
    
    #initializing the layer to the original trained weights for next round
    model.layers[-2].set_weights([W])
    
    
    

RL
221
    960/Unknown - 4s 4ms/step - loss: 0.6433 - accuracy: 0.7326/n
    960/Unknown - 5s 5ms/step - loss: 0.6174 - accuracy: 0.7411/n
0.03589547535414261
RL
511
    960/Unknown - 4s 5ms/step - loss: 0.6544 - accuracy: 0.7292/n
    960/Unknown - 5s 5ms/step - loss: 0.6426 - accuracy: 0.7262/n
0.021787259982156686
RL
435
    960/Unknown - 4s 5ms/step - loss: 0.6437 - accuracy: 0.7304/n
    960/Unknown - 4s 4ms/step - loss: 0.6159 - accuracy: 0.7372/n
0.037850377647215876
RL
481
    960/Unknown - 4s 4ms/step - loss: 0.6468 - accuracy: 0.7276/n
    960/Unknown - 4s 4ms/step - loss: 0.6038 - accuracy: 0.7406/n
0.05294672750790285
RL
25
    960/Unknown - 4s 4ms/step - loss: 0.6420 - accuracy: 0.7334/n
    960/Unknown - 4s 4ms/step - loss: 0.6107 - accuracy: 0.7432/n
0.04129306096937701
RL
426
    960/Unknown - 4s 4ms/step - loss: 0.6627 - accuracy: 0.7276/n
    960/Unknown - 4s 5ms/step - loss: 0.5933 - accuracy: 0.7451/n
0.07942963896256826
RL
101
    960/Unknown - 5s 5ms/step - loss: 

    960/Unknown - 4s 4ms/step - loss: 0.6500 - accuracy: 0.7317/n
    960/Unknown - 4s 4ms/step - loss: 0.6436 - accuracy: 0.7336/n
0.016395310894489194
RL
276
    960/Unknown - 4s 4ms/step - loss: 0.6324 - accuracy: 0.7328/n
    960/Unknown - 4s 4ms/step - loss: 0.6591 - accuracy: 0.7230/n
    960/Unknown - 4s 4ms/step - loss: 0.6554 - accuracy: 0.7254/n
0.013702960704201064
RL
459
    960/Unknown - 3s 4ms/step - loss: 0.6509 - accuracy: 0.7233/n
    803/Unknown - 3s 4ms/step - loss: 0.7844 - accuracy: 0.6755

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



    960/Unknown - 4s 4ms/step - loss: 0.6591 - accuracy: 0.7271/n
0
RL
446
    960/Unknown - 5s 5ms/step - loss: 0.6499 - accuracy: 0.7295/n
    960/Unknown - 4s 4ms/step - loss: 0.6561 - accuracy: 0.7282/n
0.003876100577593458
RL
29
    960/Unknown - 4s 4ms/step - loss: 0.6637 - accuracy: 0.7266/n
    960/Unknown - 4s 4ms/step - loss: 0.6620 - accuracy: 0.7257/n
0.011630199335825977
RL
117
    960/Unknown - 4s 4ms/step - loss: 0.6455 - accuracy: 0.7284/n
    960/Unknown - 4s 5ms/step - loss: 0.6678 - accuracy: 0.7231/n
0
RL
14
    960/Unknown - 4s 4ms/step - loss: 0.6457 - accuracy: 0.7322/n
    960/Unknown - 4s 5ms/step - loss: 0.6484 - accuracy: 0.7285/n
0.007318769266409278
RL
412
    960/Unknown - 4s 4ms/step - loss: 0.6602 - accuracy: 0.7220/n
    960/Unknown - 4s 4ms/step - loss: 0.6430 - accuracy: 0.7311/n
0.027238309091044403
RL
155
    960/Unknown - 3s 4ms/step - loss: 0.6664 - accuracy: 0.7236/n
    960/Unknown - 4s 4ms/step - loss: 0.6363 - accuracy: 0.7312/n
0.040145443968

    960/Unknown - 4s 4ms/step - loss: 0.6515 - accuracy: 0.7267/n
    960/Unknown - 4s 5ms/step - loss: 0.6506 - accuracy: 0.7251/n
0.010966013443151837
RL
248
    960/Unknown - 4s 4ms/step - loss: 0.6397 - accuracy: 0.7325/n
    960/Unknown - 4s 4ms/step - loss: 0.6469 - accuracy: 0.7268/n
0.002750279983859674
RL
389
    960/Unknown - 4s 5ms/step - loss: 0.6644 - accuracy: 0.7219/n
    960/Unknown - 4s 5ms/step - loss: 0.6736 - accuracy: 0.7222/n
0.0007660244183109646
RL
79
    960/Unknown - 4s 4ms/step - loss: 0.6547 - accuracy: 0.7244/n
    960/Unknown - 4s 4ms/step - loss: 0.6494 - accuracy: 0.7309/n
0.015343286560188676
RL
368
    960/Unknown - 4s 4ms/step - loss: 0.6499 - accuracy: 0.7296/n
    960/Unknown - 4s 5ms/step - loss: 0.6536 - accuracy: 0.7277/n
0.00630287530771966
RL
32
    960/Unknown - 4s 4ms/step - loss: 0.6511 - accuracy: 0.7300/n
    960/Unknown - 4s 5ms/step - loss: 0.6406 - accuracy: 0.7351/n
0.020519303228282208
RL
312
    960/Unknown - 4s 4ms/step - loss: 0.64

...

    960/Unknown - 4s 5ms/step - loss: 0.6590 - accuracy: 0.7246/n
0.00025204210653705615
RL
295
    960/Unknown - 4s 4ms/step - loss: 0.6505 - accuracy: 0.7294/n
    960/Unknown - 4s 4ms/step - loss: 0.6284 - accuracy: 0.7365/n
0.03213555322353689
RL
241
    960/Unknown - 5s 5ms/step - loss: 0.6607 - accuracy: 0.7258/n
    960/Unknown - 4s 4ms/step - loss: 0.6664 - accuracy: 0.7254/n
0.004240418044501029
RL
294
    960/Unknown - 4s 4ms/step - loss: 0.6474 - accuracy: 0.7276/n
    960/Unknown - 4s 5ms/step - loss: 0.6481 - accuracy: 0.7312/n
0.009264607637329938
RL
467
    960/Unknown - 4s 5ms/step - loss: 0.6504 - accuracy: 0.7272/n
    960/Unknown - 5s 5ms/step - loss: 0.6472 - accuracy: 0.7311/n
0.013260272334591696
RL
382
    960/Unknown - 4s 5ms/step - loss: 0.6527 - accuracy: 0.7304/n
    960/Unknown - 4s 4ms/step - loss: 0.6506 - accuracy: 0.7283/n
0.012117065506657467
RL
505
    960/Unknown - 4s 4ms/step - loss: 0.6573 - accuracy: 0.7272/n
    960/Unknown - 4s 5ms/step - loss: 0

    960/Unknown - 4s 5ms/step - loss: 0.6523 - accuracy: 0.7295/n
    960/Unknown - 3s 3ms/step - loss: 0.6598 - accuracy: 0.7254/n
0.0024765610569010745
RL
279
    960/Unknown - 5s 5ms/step - loss: 0.6320 - accuracy: 0.7312/n
    960/Unknown - 3s 4ms/step - loss: 0.6558 - accuracy: 0.7231/n
0
RL
244
    960/Unknown - 4s 4ms/step - loss: 0.6394 - accuracy: 0.7351/n
    960/Unknown - 4s 4ms/step - loss: 0.6617 - accuracy: 0.7217/n
0
RL
266
    960/Unknown - 4s 4ms/step - loss: 0.6507 - accuracy: 0.7314/n
    960/Unknown - 4s 4ms/step - loss: 0.6487 - accuracy: 0.7317/n
0.011929841496651874
RL
354
    960/Unknown - 4s 4ms/step - loss: 0.6544 - accuracy: 0.7302/n
    960/Unknown - 3s 3ms/step - loss: 0.6573 - accuracy: 0.7311/n
0.0070188162180421145
RL
41
    960/Unknown - 3s 3ms/step - loss: 0.6611 - accuracy: 0.7304/n
    960/Unknown - 4s 4ms/step - loss: 0.6498 - accuracy: 0.7312/n
0.02125150712759937
RL
437
    960/Unknown - 3s 3ms/step - loss: 0.6530 - accuracy: 0.7290/n
    960/Unkn

    960/Unknown - 3s 3ms/step - loss: 0.6491 - accuracy: 0.7296/n
0.012667466919835326
RL
283
    960/Unknown - 3s 3ms/step - loss: 0.6429 - accuracy: 0.7327/n
    960/Unknown - 4s 4ms/step - loss: 0.6551 - accuracy: 0.7253/n
0
RL
278
    960/Unknown - 3s 3ms/step - loss: 0.6647 - accuracy: 0.7292/n
    960/Unknown - 4s 4ms/step - loss: 0.6517 - accuracy: 0.7303/n
0.023021976827197525
RL
239
    960/Unknown - 4s 5ms/step - loss: 0.6417 - accuracy: 0.7331/n
    960/Unknown - 4s 4ms/step - loss: 0.6677 - accuracy: 0.7229/n
0
RL
88
    960/Unknown - 3s 3ms/step - loss: 0.6540 - accuracy: 0.7281/n
    960/Unknown - 3s 3ms/step - loss: 0.6507 - accuracy: 0.7287/n
0.013321749095266613
RL
398
    960/Unknown - 4s 4ms/step - loss: 0.6675 - accuracy: 0.7248/n
    960/Unknown - 3s 4ms/step - loss: 0.6412 - accuracy: 0.7312/n
0.036291345432158166
RL
186
    960/Unknown - 4s 4ms/step - loss: 0.6605 - accuracy: 0.7278/n
    960/Unknown - 3s 4ms/step - loss: 0.6606 - accuracy: 0.7243/n
0.00987850441

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



    960/Unknown - 3s 3ms/step - loss: 0.6438 - accuracy: 0.7340/n
    960/Unknown - 4s 5ms/step - loss: 0.6653 - accuracy: 0.7281/n
0
RL
113
    960/Unknown - 4s 4ms/step - loss: 0.6555 - accuracy: 0.7241/n
    960/Unknown - 4s 4ms/step - loss: 0.6395 - accuracy: 0.7331/n
0.025985362617235823
RL
319
    960/Unknown - 4s 4ms/step - loss: 0.6547 - accuracy: 0.7239/n
    960/Unknown - 3s 4ms/step - loss: 0.6578 - accuracy: 0.7234/n
0.006984659560500086
RL
507
    960/Unknown - 4s 4ms/step - loss: 0.6538 - accuracy: 0.7248/n
    960/Unknown - 4s 4ms/step - loss: 0.6392 - accuracy: 0.7309/n
0.024673584508273054
RL
342
    960/Unknown - 4s 4ms/step - loss: 0.6432 - accuracy: 0.7323/n
    960/Unknown - 4s 4ms/step - loss: 0.6744 - accuracy: 0.7252/n
0
RL
494
    960/Unknown - 3s 3ms/step - loss: 0.6445 - accuracy: 0.7284/n
    960/Unknown - 4s 4ms/step - loss: 0.6423 - accuracy: 0.7342/n
0.01214889286216935
RL
313
    960/Unknown - 4s 5ms/step - loss: 0.6489 - accuracy: 0.7298/n
    960/Unkno

In [62]:
n

array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4.

In [63]:
mu

array([0.00888938, 0.01610061, 0.00837883, 0.01453744, 0.00421896,
       0.01008685, 0.01634552, 0.01765314, 0.00863036, 0.02229861,
       0.0090888 , 0.00950271, 0.01923995, 0.01765045, 0.0168507 ,
       0.01037557, 0.01508428, 0.02187748, 0.00285196, 0.01236071,
       0.01787627, 0.01182208, 0.0142309 , 0.01511085, 0.01275801,
       0.04569408, 0.00454868, 0.02376892, 0.01319489, 0.01435445,
       0.01350186, 0.01136536, 0.01836858, 0.01426629, 0.00464154,
       0.01400211, 0.0048877 , 0.02427424, 0.01098361, 0.03639021,
       0.00953095, 0.01013067, 0.00616416, 0.01318619, 0.01233909,
       0.00734317, 0.01792942, 0.01038419, 0.00760724, 0.00602574,
       0.0125174 , 0.01499379, 0.00410682, 0.        , 0.0064774 ,
       0.0089462 , 0.01070628, 0.01132952, 0.00447797, 0.00374683,
       0.01718916, 0.00466974, 0.01003945, 0.00655572, 0.0077806 ,
       0.0161892 , 0.01071709, 0.00676884, 0.01293556, 0.01037902,
       0.01592078, 0.00641413, 0.00645069, 0.01044299, 0.00723

In [69]:
a = np.array([1, 3, 6, 3, 7])
p=np.sqrt(2*np.log10(20)/a)
m = np.ones(5)

In [70]:
p+m

array([2.6130902 , 1.93131806, 1.65854132, 1.93131806, 1.60969079])

In [67]:
mu[argmax(mu)]

NameError: name 'argmax' is not defined